# First data exploration

### Extracting and cleaning plot summaries 

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from transformers import pipeline
import numpy as np
from scipy.special import softmax


import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

#Setting pandas display options
pd.set_option("max_colwidth", None)


#Src folder path
src_folder = 'src/data/'

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\march\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\march\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [9]:
df_plot_summaries = pd.read_csv(src_folder + 'plot_summaries.txt', sep='\t', header=None,  names=['id', 'summary'])
df_plot_summaries.sample(2)

,id,summary
12594,3195590,"The story is on the concept of karma and bad deeds. It follows Viren Sahai , underworld don who rules over India. His son Abhir is spoilt and believes he can do anything due to his father's power. When Viren's enemy Khan attacks Abhir, Viren demands revenge. The plot then moves onto the life of a simple cook, Shekhar Bhatia who is married to Dr. Kavita . Shekhar lives with his parents, wife and his adorable younger brother Aryan , who is just as spoilt as Abhir. When Aryan runs away from home, Shekhar searches the whole city to find him. During his search, he witnesses Viren attacking a cinema theater as Khan is inside it, and Shekhar begins to help the innocent people who are being shot in the theater. He helps them out to the hospital, and as Viren sees him helping the people, he believes Shekhar is part of Khan's gang, and Viren shoots Shekhar however he survives. Viren also shoots Khan's nephew, and as Khan escapes, Viren also flees. As Shekhar, who is in critical condition, is trying to reach home to his family before he dies, he is run over by Viren's car accidentally and Shekhar dies. When Aryan arrives home, he realizes Viren has murdered Shekhar, and he loses his temper. Together, Aryan and his group of friends form a gang and kidnap Viren's family including Abhir. However, Abhir escapes and reaches to his father. Since the police decide to help Viren, Aryan's plan fails. Aryan tries not to do harm to any of his family member as he knows how precious they are, though Viren's wife suddenly dies of suffercation. Viren believes Aryan is responsible for this, and takes his whole gang, and starts a huge war with Aryan and his gang. The war breaks out, and Khan enters. As Viren murders Khan, he realizes that his son Abhir was part of Khan's gang all along. Abhir tries to shoot his father Viren, but Aryan rescues Viren and kills Abhir. He leaves the scene, and as Viren demands Aryan to kill him now, Aryan ends the film on a note on karma, and explains his punishment is not death, but life as he has nothing to live for. In the end, Aryan becomes a cook like his late brother and him and his friends start to work in the same canteen Shekhar used to work in."
15078,4018828,"Bob Munro , a successful California beverage company executive, is struggling with a dysfunctional family, with his materialistic wife Jamie , his spoiled, sharp-tongued teenage daughter Cassie ([[JoJo , and young son Carl an adolescent weightlifter who likes Hip hop. At a company picnic, Bob is embarrassed in front of his self-absorbed boss Todd by Cassie's militant friend Gretchen, who hurls a tub of disgusting slime over Todd. Looking forward to a big family vacation in Hawaii, Bob is punished by Todd by having to present a merger proposal to the grassroots Alpine Soda company in Boulder, Colorado instead. Concealing the real reason for going to Boulder, Bob rents an RV from the dodgy dealer Irv and tells his family they are traveling to the Rockies. On their trip, Bob and his family encounter many mishaps. These include Bob damaging the parking brake, crashing into and running over objects such as shopping carts, flushing out a trio of raccoons with a stink bomb, and fixing a clogged sewage system. Along the way, the Munros meet another traveling family, the Gornickes, consisting of Travis , Mary Jo , and their sons and daughter, Earl , Billy ""Not Earl"" , and Moon . Earl develops a romantic interest in Cassie and Carl starts to like Moon, but thinking that the Gornickes are too strange for them, the adult Munros decide to ditch them; when the Gornickes reappear at another stop, the Munros believe they are stalking them. Meanwhile, to disguise his business trip, Bob tries to e-mail a proposal outline from his laptop, working in restrooms; eventually, a hitchhiker steals Bob's laptop, leaving him with only a BlackBerry PDA, which he does manage to use to compose and wirelessly send his proposal to his company. The Gornickes then

Some cleaning is necessary for the plot summaries. There appears to be in the data:

1) {{annotation}}
2) links (references to wikipedia)
3) <> html delimiters
4) ... further inspection needed

That need to be removed


In [10]:
def clean_plot(txt):

    #Remove URLs
    txt = re.sub(r"http\S+|www\.\S+", '', txt)

    #Remove HTML tags
    txt = re.sub(r'<.*?>', '', txt)

    #Remove {{annotations}}
    txt = re.sub(r'\{\{.*?\}\}', '', txt)

    #Remove the ([[ annotation that is never closed
    txt = re.sub(r'\(\[\[', '', txt)

    #Remove the synopsis from context
    txt = re.sub(r'Synopsis from', '', txt)

    #Remove <ref...}} tags
    txt = re.sub(r'<ref[^}]*}}', '', txt)

    return txt

In [5]:
df_test_clean = df_plot_summaries.copy()
df_test_clean["summary"] = df_plot_summaries['summary'].apply(clean_plot)
df_test_clean.sample(2)

,id,summary
9885,27768311,"Hanae and Chōko are two hard-working sisters living in the countryside with their father. Hanae runs a snack shop and Chōko works in a hair salon. Teruhiko, Hanae's first boyfriend from high school who now works for a large company in Tokyo, visits the snack shop one day. They become reacquainted and Teruhiko wishes to marry Hanae. Romantic complications ensue when he meets Chōko."
39947,17038281,"The film follows a young woman, Wu Hongyan Liu Dan , who works as a prison guard who aids in the execution of female prisoners. Lonely and widowed, Wu finds herself taking the night train to a dating service in a neighboring city. After a series of unsuccessful dates, she meets Li Jun Qi Dao , with whom she begins a relationship. It soon becomes clear that Li Jun is hiding a secret: that he is the widower of one of Wu's executed prisoners. Li Jun is torn both by his attraction to Wu, but also his desire to exact some sort of vengeance. Wu, meanwhile, must consider her own safety in this new volatile relationship."


#### Sentiment analysis test

I see two possibilities: making classical sentiment analysis using twitter-roberta-base-sentiment trained on tweets or using distilbert-base-uncased-emotions for emotions

In [ ]:
#First possibility: sentence by sentence sentiment classification

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)



dict_labels = {0: 'negative', 1: 'neutral', 2: 'positive'}
for t in df_test_clean['summary'].sample(1):
    for sentence in nltk.sent_tokenize(t):
        print(sentence)
        t_encoded = tokenizer(sentence, return_tensors='pt')
        t_output = model(**t_encoded)
        t_scores = softmax(t_output.logits.detach().numpy(), axis=1)
        t_predicted = np.argmax(t_scores)
        print(dict_labels[t_predicted], t_scores[0][t_predicted])
    

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


A senator's grandson is murdered while incarcerated at California's notorious "Doscher State Prison", nicknamed "Dachau" by convicts for the number of unsolved murders of inmates that have taken place therein.
negative 0.78488505
Hired by the senator, Gulf War veteran John Woodrow  infiltrates the prison to find the murderer, and learns that the grandson was killed by corrupt head guard Lt. Hopkins .
negative 0.5758992
When Hopkins discovers Woodrow's true identity, he has him tortured by inamates and guards.
neutral 0.595048
After an arduous shooting chase through the desert, Woodrow escapes.
neutral 0.74044


In [ ]:
# Second possibility: sentence to sentence emotion classification

#the emotions are anger, fear, joy, love, sadness, surprise and neutral

classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

for t in df_test_clean['summary'].sample(1):
    for sentence in nltk.sent_tokenize(t):
        print(sentence)
        out = classifier(sentence)[0]
        best_emotion_dict = max(out, key=lambda x: x['score'])
        best_label = best_emotion_dict['label']
        best_score = best_emotion_dict['score']
        print(best_label, best_score)    

The story is about a lawsuit concerning injuries caused by a defective automobile.
neutral 0.6862418055534363
The suit takes on a personal dimension because the injured plaintiff's attorney, Jedediah Tucker Ward  discovers that the automobile manufacturer's attorney Maggie Ward  is his estranged daughter.
sadness 0.7743082046508789
The central premise of the film is roughly analogous to the controversy surrounding the Ford Pinto.
neutral 0.7334937453269958
Jedediah Ward is a liberal civil rights lawyer who has based his career on helping people avoid being taken for a ride by the rich and powerful; he's pursued principle at the expense of profit, though he has a bad habit of not following up on his clients after their cases are settled.
neutral 0.6313906908035278
Jed's daughter, Maggie, has had a bad relationship with her father ever since she discovered that he was cheating on her mother, Estelle , and while she also has made a career in law, she has taken a very different professiona